In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import math
import os
import sys
from ggw import *
from scipy.spatial import ConvexHull
from matplotlib.lines import Line2D
%matplotlib widget
%matplotlib inline
# !pip install geomloss
# !pip install scikit-learn

# Preprocess

In [ ]:
data_2_path = '/Users/a1/Tugas Vera/Year 4 Fall/MATH 4995/MNIST Sign Language/MNIST_number/number_2.csv'
data_5_path = '/Users/a1/Tugas Vera/Year 4 Fall/MATH 4995/MNIST Sign Language/MNIST_number/number_5.csv'
data_0_path = '/Users/a1/Tugas Vera/Year 4 Fall/MATH 4995/MNIST Sign Language/MNIST_number/number_0.csv'
data_8_path = '/Users/a1/Tugas Vera/Year 4 Fall/MATH 4995/MNIST Sign Language/MNIST_number/number_8.csv'


df_2 = pd.read_csv(data_2_path,encoding='latin-1', header=None) 
# print(df_2.head(10))

df_5 = pd.read_csv(data_5_path,encoding='latin-1', header=None) 
# print(df_5.head(10))

df_0 = pd.read_csv(data_0_path,encoding='latin-1', header=None) 
print(df_0.head(10))

df_8 = pd.read_csv(data_8_path,encoding='latin-1', header=None) 
print(df_8.head(10))

In [ ]:
df_2.iloc[0,0] = 10
df_2.columns = ['x','y']
df_2['x'] = df_2['x'].astype(int)
df_2.dtypes

df_5.iloc[0,0] = 6
df_5.columns = ['x','y']
df_5['x'] = df_5['x'].astype(int)
df_5.dtypes

df_0.iloc[0,0] = 18
df_0.columns = ['x','y']
df_0['x'] = df_0['x'].astype(int)
df_0.dtypes

df_8.iloc[0,0] = 15
df_8.columns = ['x','y']
df_8['x'] = df_8['x'].astype(int)
df_8.dtypes
print(df_2.shape)
print(df_5.shape)

In [ ]:
df_2_chosen = df_2.head(270)
print(df_2_chosen.shape)

df_5_chosen = df_5.head(270)
print(df_5_chosen.shape)

In [ ]:
plt.figure(figsize=(8, 6))
# test = np.concatenate((df_train_arr, df_test_arr), axis=0)
plt.scatter(df_2_chosen['x'],df_2_chosen['y'], color='blue', marker='o', label='2')
plt.scatter(df_5_chosen['x'],df_5_chosen['y'], color='red', marker='x', label='5')
plt.title('2D Scatter Plot')
plt.xlabel('X-Axis')
plt.ylabel('Y-Axis')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
# test = np.concatenate((df_train_arr, df_test_arr), axis=0)

plt.subplot(1,2,1)
plt.scatter(df_2_chosen['x'],df_2_chosen['y'], color='blue', marker='o', label='2')
# plt.scatter(df_5_chosen['x'],df_5_chosen['y'], color='red', marker='x', label='5')
plt.title('2D Scatter Plot')
plt.xlabel('X-Axis')
plt.ylabel('Y-Axis')
plt.legend()
plt.grid(True)

ax = plt.subplot(1,2,2)
ax.scatter(df_5_chosen['x'],df_5_chosen['y'], color='red', marker='x', label='5')
ax.set_title('2D Scatter Plot')
ax.set_xlabel('X-Axis')
ax.set_ylabel('Y-Axis')
ax.legend()
ax.grid(True)
plt.show()

In [ ]:
from scipy.spatial import ConvexHull
from matplotlib.lines import Line2D


# datas = [df_jump1, df_jump2, df_punch1, df_punch2, df_sit, df_wave]

X = df_2_chosen.values
Y = df_5_chosen.values

P_ggw,logs = gw_global(X,Y,epsilon=1e-6,verbose=True,IterMax=500,log=True)

# scatter plot X and Y
plt.figure(figsize=(9.5,8))
plt.subplot(2,2,1)
plt.scatter(X[:,0], X[:,1], color='blue', label='Source', marker='.', s=10)
plt.scatter(Y[:,0], Y[:,1], color='orange', label='Target', marker='X', s=10)

# plt.tight_layout()
# plt.show()

for i in range(len(X)):
    for j in range(len(Y)):
        if P_ggw[i,j]>0:
            plt.plot([X[i][0], Y[j][0]], [X[i][1], Y[j][1]],color='gray',zorder=-4,linewidth=1,alpha=0.5)
plt.legend()
plt.yticks([])
plt.tight_layout()

# optimization plots with the bounding box and linear constraints
ax4 = plt.subplot(2,2,4)

# BOUNDING BOX
for i,E in enumerate(logs['E_cache']):
    plt.scatter(E[:,0],E[:,1],alpha=1,marker='s',facecolor='white',s=20,label=f'iter {i}',color='C0')

# c_cache.append((lower_bounds,upper_bounds)) 
for i,c in enumerate(logs['c_cache']):
    if i == 0:
        lower_bounds,upper_bounds=c
        width = upper_bounds[0] - lower_bounds[0]
        height = upper_bounds[1] - lower_bounds[1]
        plt.plot([lower_bounds[0],upper_bounds[0],upper_bounds[0],lower_bounds[0],lower_bounds[0]],[lower_bounds[1],lower_bounds[1],upper_bounds[1],upper_bounds[1],lower_bounds[1]],label='Bounding box',linewidth=0.5,color='C0')
    else:
        A,b=c
        if A[1] == 0:
            plt.plot([b,b],[lower_bounds[1]-0.1*height,upper_bounds[1]+0.1*height],linewidth=0.5,color='C0')
        else:
            x = np.array([lower_bounds[0]-0.1*width,upper_bounds[0]+0.1*width])
            y = (b - A[0] * x)/A[1]
            plt.plot(x,y,linewidth=0.5,color='C0')


plt.xlim(lower_bounds[0]-0.1*width,upper_bounds[0]+0.1*width)
plt.ylim(lower_bounds[1]-0.1*height,upper_bounds[1]+0.1*height)

for i,E0 in enumerate(logs['E0_cache']):
    plt.scatter(E0[0],E0[1],marker='x',s=30,color='red')
    plt.text(E0[0],E0[1],f'{i}',fontsize=8)    

L = logs['L']

for i,P in enumerate(logs['Pi_cache'][:-1]):
    W = np.sum(2 * X.T @ P @ Y)
    w = np.sum(L*P)
    plt.scatter(w,W,color='red',marker='>',facecolor='white',s=50)
    plt.text(w,W,f'{i}',fontsize=8)  

P = logs['Pi_cache'][-1]
W = np.sum(2 * X.T @ P @ Y)
w = np.sum(L*P)
plt.scatter(w,W,color='yellow',marker='*',facecolor='yellow',s=100,zorder=15)

Ws = [W]
ws = [w]
M = 1000
for _ in range(M):
    n = len(X)
    perm = np.random.permutation(n)
    mat = np.zeros((n, n), dtype=int)
    for i in range(n):
        mat[i, perm[i]] = 1
    P1 = mat
    i = np.random.choice(len(logs['Pi_cache']))
    P = logs['Pi_cache'][i]
    a = (np.random.choice(5)/5)**3
    Pi = (a) * P + (1-a) * P1
    W = np.sum(2 * X.T @ Pi @ Y)
    w = np.sum(L*Pi)
    Ws.append(W)
    ws.append(w)
    
for _ in range(M):
    n = len(X)
    perm = np.random.permutation(n)
    mat = np.zeros((n, n), dtype=int)
    for i in range(n):
        mat[i, perm[i]] = 1
    Pi = mat
    W = np.sum(2 * X.T @ Pi @ Y)
    w = np.sum(L*Pi)
    Ws.append(W)
    ws.append(w)
    
points = np.array(list(zip(ws, Ws)))
hull = ConvexHull(points)
for simplex in hull.simplices:
    plt.plot(points[simplex, 0], points[simplex, 1], color='k',zorder=-12,linewidth=1)
    plt.scatter(points[simplex, 0], points[simplex, 1],marker='o',s=10,zorder=-15,edgecolors=[(0, 0.5, 0, 1)],facecolors=[(0, 0.5, 0, 1)])

plt.scatter(ws[::10],Ws[::10],marker='o',s=10,zorder=-15,edgecolors=[(0, 0.5, 0, 1)],facecolors=[(0, 0.5, 0, 1)])

x = np.linspace(lower_bounds[0]-0.1*width, upper_bounds[0]+0.1*width, 400)
y = np.linspace(lower_bounds[1]-0.1*height, upper_bounds[1]+0.1*height, 400)
x, y = np.meshgrid(x, y)
z = -x-y**2

plt.contour(x, y, z, levels=20, colors='green', linewidths=0.5,zorder=-5) 
plt.contourf(x, y, z, levels=20,zorder=-10,alpha=0.5) 

plt.xlabel(r'$w = \langle L,\Gamma \rangle$')
plt.ylabel(r'$W = 2X\Gamma Y^T$')


handles = [
    Line2D([0], [0], color='C0', lw=1, label='Constraints'),
    Line2D([0], [0], color='k', lw=1, label='Convex hull'),
    Line2D([0], [0], marker='s',lw=0,color='C0',markerfacecolor='white',markersize=5, label='Extreme point'),
    Line2D([0], [0], marker='>',lw=0,color='red',markersize=6, markerfacecolor='white',label='Upper bound'),
    Line2D([0], [0], marker='x',lw=0,color='red',markersize=6, label='Lower bound'),
    Line2D([0], [0], marker='o',lw=0,color='darkgreen',markersize=3, label='Random $\Gamma$'),
    Line2D([0], [0], marker='*',lw=0,color='yellow',markersize=10, label='Global optimum')
]

ax4.legend(handles=handles,loc='upper left', bbox_to_anchor=(1,1),frameon=False)
plt.tight_layout()

plt.subplot(2,2,3)
plt.plot(range(logs['niter']),logs['u_cache'],label='Upper bound')
plt.plot(range(logs['niter']),logs['l_cache'],label='Lower bound')
plt.xticks(range(logs['niter']))
           
plt.xlabel('Iteration')
plt.legend()
plt.tight_layout()

ax2 = plt.subplot(2,2,2)
plt.imshow(P_ggw)
plt.xticks(range(0,n,2))
plt.title("$\Gamma^*$")
ax2.set_aspect('auto')
plt.tight_layout()

# Performance Comparison between GGW (500 iter) and GW (5000 iter)

In [ ]:
X = df_2_chosen.values
Y = df_5_chosen.values

P_ggw,logs = gw_global(X,Y,epsilon=1e-6,verbose=False,log=True)

p = ot.unif(len(X))
q = ot.unif(len(Y))
C1 = ot.dist(X,X)
C2 = ot.dist(Y,Y)
P_gw = ot.gromov.gromov_wasserstein(C1, C2, p, q, 'square_loss', verbose=False, log=False,max_iter=5000)            # Shows that it does not converge in reasonable time

plt.figure(figsize=(8,8))
plt.subplot(2,2,1)
plt.imshow(P_ggw)


plt.subplot(2,2,2)
plt.imshow(P_gw)

plt.subplot(2,2,3)

plt.scatter(X[:,0],X[:,1],alpha=1)
plt.scatter(Y[:,0],Y[:,1],alpha=1)

P = P_ggw

for i in range(len(X)):
    for j in range(len(Y)):
        if P[i,j]>0:
            plt.plot([X[i,0],Y[j,0]],[X[i,1],Y[j,1]],linewidth=P[i,j]/np.max(P),color='grey',zorder=-1)
            
ax = plt.gca()
ax.set_aspect('equal')

plt.subplot(2,2,4)

plt.scatter(X[:,0],X[:,1],alpha=1)
plt.scatter(Y[:,0],Y[:,1],alpha=1)

P = P_gw

for i in range(len(X)):
    for j in range(len(Y)):
        if P[i,j]>0:
            plt.plot([X[i,0],Y[j,0]],[X[i,1],Y[j,1]],linewidth=P[i,j]/np.max(P),color='grey',zorder=-1)
            
ax = plt.gca()
ax.set_aspect('equal')

print(f'Loss(P_ggw) - Loss(P_gw): {gw_obj(C1,C2,P_ggw)-gw_obj(C1,C2,P_gw/np.max(P_gw)):.4e}')

# K-means with Gromov-Wasserstein Distance

In [37]:
df_0_chosen = df_0.head(270)
df_8_chosen = df_8.head(270)

additional data

In [38]:
# df_2_chosen.head(47)

In [39]:
data_2_path_2 = '/Users/a1/Tugas Vera/Year 4 Fall/MATH 4995/MNIST Sign Language/MNIST_number/number_2_2.csv'
df_2_2 = pd.read_csv(data_2_path_2,encoding='latin-1', header=None) 
df_2_2.iloc[0,0] = 5
df_2_2.columns = ['x','y']
df_2_2['x'] = df_2_2['x'].astype(int)
df_2_2_chosen = df_2_2.head(270)
# df_2_2_chosen.tail(5)

In [40]:
data_2_path_3 = '/Users/a1/Tugas Vera/Year 4 Fall/MATH 4995/MNIST Sign Language/MNIST_number/number_2_3.csv'
df_2_3 = pd.read_csv(data_2_path_3,encoding='latin-1', header=None) 
df_2_3.iloc[0,0] = 10
df_2_3.columns = ['x','y']
df_2_3['x'] = df_2_3['x'].astype(int)
df_2_3_chosen = df_2_3.head(270)
# df_2_3_chosen.tail(5)

In [41]:
data_3_path_1 = '/Users/a1/Tugas Vera/Year 4 Fall/MATH 4995/MNIST Sign Language/MNIST_number/number_3_1.csv'
df_3_1 = pd.read_csv(data_3_path_1,encoding='latin-1', header=None) 
df_3_1.iloc[0,0] = 8
df_3_1.columns = ['x','y']
df_3_1['x'] = df_3_1['x'].astype(int)
df_3_1_chosen = df_3_1.head(270)
# df_3_1_chosen.tail(5)

In [42]:
data_0_path_1 = '/Users/a1/Tugas Vera/Year 4 Fall/MATH 4995/MNIST Sign Language/MNIST_number/number_0_1.csv'
df_0_2 = pd.read_csv(data_0_path_1,encoding='latin-1', header=None) 
df_0_2.iloc[0,0] = 7
df_0_2.columns = ['x','y']
df_0_2['x'] = df_0_2['x'].astype(int)
df_0_2_chosen = df_0_2.head(270)
# df_0_2_chosen.tail(5)

In [43]:
data_3_path_2 = '/Users/a1/Tugas Vera/Year 4 Fall/MATH 4995/MNIST Sign Language/MNIST_number/number_3_2.csv'
df_3_2 = pd.read_csv(data_3_path_2,encoding='latin-1', header=None) 
df_3_2.iloc[0,0] = 17
df_3_2.columns = ['x','y']
df_3_2['x'] = df_3_2['x'].astype(int)
df_3_2_chosen = df_3_2.head(270)
# df_3_2_chosen.tail(5)

K-Means

In [ ]:
import torch
from geomloss import SamplesLoss

# Define the two metric measure spaces (X, Y) as torch tensors
X = torch.tensor(df_2_chosen.values)  # Data points in metric space X
Y1 = torch.tensor(df_5_chosen.values)  # Data points in metric space Y
Y2 = torch.tensor(df_0_chosen.values)
Y3 = torch.tensor(df_8_chosen.values)
Y4 = torch.tensor(df_2_2_chosen.values)
Y5 = torch.tensor(df_2_3_chosen.values)
Y6 = torch.tensor(df_3_1_chosen.values)
Y7 = torch.tensor(df_0_2_chosen.values)
Y8 = torch.tensor(df_3_2_chosen.values)

X = X.float()
Y1 = Y1.float()
Y2 = Y2.float()
Y3 = Y3.float()
Y4 = Y4.float()
Y5 = Y5.float()
Y6 = Y6.float()
Y7 = Y7.float()
Y8 = Y8.float()

target = X
test = [X,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8]
test_values = ['2','5','0','8','2_2','2_3','3','0_2','3_2']

def cost(x, y):
    return torch.cdist(x, y, p=2)  # Euclidean distance

loss = SamplesLoss("sinkhorn", p=2, blur=0.05)
gw_distances = []

for index,val in enumerate(test):
    gw_dist = loss(X, val)
    print(f"Gromov-Wasserstein distance (X, {test_values[index]}): ", gw_dist.item())
    gw_distances.append([test_values[index],gw_dist.item()])

print(gw_distances)


In [ ]:
from sklearn.cluster import KMeans

# kaloga coba masukkin 2 yg lain, jgn cuma pake 2 yg sama aja

gw_distances_np = np.array([dist[1] for dist in gw_distances])
X_features = gw_distances_np.reshape(-1, 1)
k = 2

kmeans = KMeans(n_clusters=k, random_state=12)      # random_state=5 kalo gapake angka 3
kmeans.fit(X_features)

cluster_labels = kmeans.labels_
print("Cluster Labels:", cluster_labels)
print("Data Labels", test_values)

In [ ]:
print("Cluster Labels:")
for i, label in enumerate(cluster_labels):
    print(f"Data Point {i}: Cluster {label}")

print("\nData Labels:")
for i, value in enumerate(test_values):
    print(f"Data Point {i}: Label {value}")